<a href="https://colab.research.google.com/github/Fizza-Rubab/CS432-GPU/blob/main/Week4/MatrixMultiplication_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-njazmuqz
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-njazmuqz
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=96d30738bafa5cc23c9241971e83843f2129eaba2958e11afce17e3b9d6358b6
  Stored in directory: /tmp/pip-ephem-wheel-cache-rapev9ho/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin
created output directory at /content/src
Out bin /content/result.out


In [28]:
%%cu
#include <stdio.h>
#include <cuda.h>
 
__global__ void MatrixMultiply(int* A, int* B, int* C, const int N) {
	//Calculate the row # 
	int Row = blockIdx.y*blockDim.y + threadIdx.y;

	//Calculate the column # 
	int Col = blockIdx.x*blockDim.x + threadIdx.x;

	if ((Row < N) && (Col < N)) {
			int sum = 0;
			// do the inner product
			for (int i=0; i<N;i++){
					sum+= A[Row*N+Col]*B[Row*N+Col];
			}
		 C[Row*N+Col] = sum;
	}
}


int main(int argc, char** argv)
{
	const int N = 3; 
	size_t size = N * N * sizeof(int);
	int *a = (int *)malloc(size);
	int *b = (int *)malloc(size);
	int *c = (int *)malloc(size);
	int *a_d, *b_d, *c_d;
	
	for(int i=0;i<N*N;++i) {
		a[i] = 1;
		b[i] = 1;
	}
	printf("Matrix A\n");
	for(int i=0;i<N;++i) {
			for(int j=0;j<N;++j){
				printf("%d ", a[i*N+j]);
			}
			printf("\n");
	}

	printf("--------------------------------------\n");
	printf("Matrix B\n");
	for(int i=0;i<N;++i) {
			for(int j=0;j<N;++j){
				printf("%d ", b[i*N+j]);
			}
			printf("\n");
	}

	//allocate device memory 
	cudaMalloc((void**)&a_d,  size);
	cudaMalloc((void**)&b_d,  size);
	cudaMalloc((void**)&c_d,  size);
	
	//copy host data to device memory
	cudaMemcpy(a_d, a,  size, cudaMemcpyHostToDevice); 
	cudaMemcpy(b_d, b,  size, cudaMemcpyHostToDevice);
	
	int BLOCK_SIZE = 16;
	int BLOCK_NUM = ceil(N/float(BLOCK_SIZE));
	dim3 dimGrid(BLOCK_NUM, BLOCK_NUM);
	dim3 dimBlock(BLOCK_SIZE, BLOCK_SIZE);
	MatrixMultiply<<<dimGrid, dimBlock>>>(a_d, b_d, c_d, N);

		//copy result from device to host
	cudaMemcpy(c, c_d,  size, cudaMemcpyDeviceToHost);

	printf("--------------------------------------\n");
	printf("Matrix A*B calculation result: \n");
	for(int i=0;i<N;++i) {
			for(int j=0;j<N;++j){
				printf("%d ", c[i*N+j]);
				}
			printf("\n");
	}
	printf("--------------------------------------\n");


	//delete data allocated on device
	cudaFree(a_d);
	cudaFree(b_d);
	cudaFree(c_d);

	//delete host memory
	free(a);
	free(b);
	free(c);

	return 0;
}

Matrix A
1 1 1 
1 1 1 
1 1 1 
--------------------------------------
Matrix B
1 1 1 
1 1 1 
1 1 1 
--------------------------------------
Matrix A*B calculation result: 
3 3 3 
3 3 3 
3 3 3 
--------------------------------------

